In [2]:
from __future__ import division
import json
import os
import numpy as np
from skimage import color as skcolor
from skimage import io as skio
import matplotlib.pyplot as plt
from scipy.misc import imresize
from mpl_toolkits.mplot3d import Axes3D
import warnings
import functools
import itertools

### CONSTANTS

MATERIAL_1 = 0
MATERIAL_2 = 1

class GCommand(object):
    """Class representing a single action of the a microvalve"""

    def __init__(self, x, y, e, material, usecs=100):
        """
        Init

        x, x location in gcode coords
        y, y location in gcode coords
        material, material indicator
        usecs, delay after movement
        """
        self.x = x
        self.y = y
        self.e = e
        self.material = material
        self.usecs = usecs
    
    def __str__(self):
        """Returns gcode representation of command"""
        return "G0 X{} Y{} E{} ;material: {}\n"\
        .format(self.x, self.y, self.e, self.material)


class TCommand(object):
    """Switch extruders"""

    def __init__(self, material):
        self.material = material
    
    def __str__(self):
        """Returns gcode representation of command"""
        return "T{};\n".format(self.material)

# x = 50, y = 50. top left corner
# outer box is 10 mm by 10 mm
# each unit is .3mm. each box separated by 1.5mm.

def convert_to_gcode(material, usecs=600, start_x=50, start_y=50, start_e=0, height=10, width=10):
    gcommands = []
    currx, curry, curre = start_x, start_y, start_e
    # starting at top left
    gcommands.append(GCommand(start_x, start_y, start_e, material))
    # top left to top right
    gcommands.append(GCommand(start_x + width, start_y, start_e + width, material))
    # top right to bottom right
    gcommands.append(GCommand(start_x + width, start_y - height, start_e + height, material))
    # bottom right to bottom left
    gcommands.append(GCommand(start_x, start_y - height, start_e + width, material))
    # bottom left to top left
    gcommands.append(GCommand(start_x, start_y, start_e + height, material))
    return gcommands

def write_gcode(gcommands, gcode_path):
    """
    Convert list of gcommands into .gcode file. Also add start and end commands

    gcommand_layers, list of lists of GCommand objects
    gcode_path, path to write output
    layer_names, names for each layer, defaults to one index naming
    heatbed_temp, start temp UNUSED
    """

    start_gcode = 'M42 P4 S250\n'
    end_gcode = 'M42 P4 S255\n'

    with open(gcode_path, 'w') as gcode_file:
        gcode_file.write(start_gcode)
        
        for gcommand in gcommands:
            gcode_file.write(str(gcommand))
        
        gcode_file.write(end_gcode)

# concentric squares
commands = []
width, height = 34, 34 # bc 10/.3 mm = 33.333333... steps
startx, starty = 50, 50
# -5 steps from x, y for the inner box
for i in range(2):
    for material in range(2):
        commands.append(TCommand(material))
        commands += convert_to_gcode(material=material, usecs=600, start_x=startx, start_y=starty, height=height, width=width)
        
        
        startx += 5
        starty -= 5
        width -= 10
        height -= 10
        
write_gcode(commands, 'extrusion-test1.gcode')

# filled square
commands = []
width, height = 34, 34 # bc 10/.3 mm = 33.333333... steps
startx, starty = 50, 50
# -1 steps from x, y for the inner box
for i in range(17):
    commands.append(TCommand(material))
    commands += convert_to_gcode(material=material, usecs=600, start_x=startx, start_y=starty, height=height, width=width)

    startx += 1
    starty -= 1
    width -= 2
    height -= 2
        
write_gcode(commands, 'extrusion-test2.gcode')